In [1]:
import re, collections
text = """The aims for this subject is for students to develop an understanding of the main 
algorithms used in naturallanguage processing, for use in a diverse range of applications 
including text classification, machine translation, and question answering. Topics to be covered 
include part-of-speech tagging, n-gram language modelling, syntactic parsing and deep learning. 
The programming language used is Python, see for more information on its use in the workshops, 
assignments and installation at home."""

In [4]:
def get_vocab(text):
    """统计词频"""
    vocab = collections.defaultdict(int)
    for word in text.strip().split():
        vocab[' '.join(word) + ' </w>'] += 1
    return vocab
print(get_vocab(text))

defaultdict(<class 'int'>, {'T h e </w>': 2, 'a i m s </w>': 1, 'f o r </w>': 4, 't h i s </w>': 1, 's u b j e c t </w>': 1, 'i s </w>': 2, 's t u d e n t s </w>': 1, 't o </w>': 2, 'd e v e l o p </w>': 1, 'a n </w>': 1, 'u n d e r s t a n d i n g </w>': 1, 'o f </w>': 2, 't h e </w>': 2, 'm a i n </w>': 1, 'a l g o r i t h m s </w>': 1, 'u s e d </w>': 2, 'i n </w>': 3, 'n a t u r a l l a n g u a g e </w>': 1, 'p r o c e s s i n g , </w>': 1, 'u s e </w>': 2, 'a </w>': 1, 'd i v e r s e </w>': 1, 'r a n g e </w>': 1, 'a p p l i c a t i o n s </w>': 1, 'i n c l u d i n g </w>': 1, 't e x t </w>': 1, 'c l a s s i f i c a t i o n , </w>': 1, 'm a c h i n e </w>': 1, 't r a n s l a t i o n , </w>': 1, 'a n d </w>': 3, 'q u e s t i o n </w>': 1, 'a n s w e r i n g . </w>': 1, 'T o p i c s </w>': 1, 'b e </w>': 1, 'c o v e r e d </w>': 1, 'i n c l u d e </w>': 1, 'p a r t - o f - s p e e c h </w>': 1, 't a g g i n g , </w>': 1, 'n - g r a m </w>': 1, 'l a n g u a g e </w>': 2, 'm o d e l l

In [5]:
"""
这个函数遍历词汇表中的所有单词，并计算彼此相邻的⼀对标记。
EXAMPLE:
    word = 'T h e <\w>'
    这个单词可以两两组合成：
        [('T', 'h'), ('h', 'e'), ('e', '<\w>')]    
输⼊:
    vocab: Dict[str, int]  # vocab统计了词语出现的词频 
输出:
    pairs: Dict[Tuple[str, str], int] # 字⺟对，pairs统计了单词对出现的频率
"""
def get_stats(vocab):

    pairs = collections.defaultdict(int)

    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += freq
    return pairs

In [6]:
"""
EXAMPLE:
    word = 'T h e <\w>'
    pair = ('e', '<\w>')
    word_after_merge = 'T h e<\w>'    
输⼊:
    pair: Tuple[str, str] # 需要合并的字符对    
    v_in: Dict[str, int]  # 合并前的vocab   
输出:
    v_out: Dict[str, int] # 合并后的vocab    
注意:
    当合并word 'Th e<\w>'中的字符对('h', 'e')时，'Th'和'e<\w>'字符对不能被合并。
"""
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair)) # 创建正则表达式,escape用于转义特殊字符
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') # 创建正则表达式对象

    for v in v_in:
        new = p.sub(''.join(pair), v) # 替换匹配到的字符为连接后的pair
        v_out[new] = v_in[v]
    return v_out

In [8]:
# 统计合并后，token的频率
def get_tokens(vocab):
    tokens = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens[token] += freq
    return tokens

In [11]:
vocab = get_vocab(text)
print("Vocab = ", vocab)
print('=' * 20)
print('Tokens before BPE:')
tokens = get_tokens(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))
print('=' * 20)
num_merges = 100
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    new_token = ''.join(best)
    vocab = merge_vocab(best, vocab)
    print('Iteration {}:'.format(i + 1))
    print('Best pair: {}'.format(best))
    tokens[new_token] = pairs[best]
    tokens[best[0]] -= pairs[best]
    tokens[best[1]] -= pairs[best]
    print('Tokens: {}'.format(tokens))
    print('Number of tokens: {}'.format(len(tokens)))
    print('=' * 20)
    print('vocab', vocab)


Vocab =  defaultdict(<class 'int'>, {'T h e </w>': 2, 'a i m s </w>': 1, 'f o r </w>': 4, 't h i s </w>': 1, 's u b j e c t </w>': 1, 'i s </w>': 2, 's t u d e n t s </w>': 1, 't o </w>': 2, 'd e v e l o p </w>': 1, 'a n </w>': 1, 'u n d e r s t a n d i n g </w>': 1, 'o f </w>': 2, 't h e </w>': 2, 'm a i n </w>': 1, 'a l g o r i t h m s </w>': 1, 'u s e d </w>': 2, 'i n </w>': 3, 'n a t u r a l l a n g u a g e </w>': 1, 'p r o c e s s i n g , </w>': 1, 'u s e </w>': 2, 'a </w>': 1, 'd i v e r s e </w>': 1, 'r a n g e </w>': 1, 'a p p l i c a t i o n s </w>': 1, 'i n c l u d i n g </w>': 1, 't e x t </w>': 1, 'c l a s s i f i c a t i o n , </w>': 1, 'm a c h i n e </w>': 1, 't r a n s l a t i o n , </w>': 1, 'a n d </w>': 3, 'q u e s t i o n </w>': 1, 'a n s w e r i n g . </w>': 1, 'T o p i c s </w>': 1, 'b e </w>': 1, 'c o v e r e d </w>': 1, 'i n c l u d e </w>': 1, 'p a r t - o f - s p e e c h </w>': 1, 't a g g i n g , </w>': 1, 'n - g r a m </w>': 1, 'l a n g u a g e </w>': 2, 'm 

In [16]:
def get_tokens_from_vocab(vocab):
    tokens_frequencies = collections.defaultdict(int)
    vocab_tokenization = {}
    for word, freq in vocab.items():
        # 看vocabulary⾥⾯的token频率，相当于上⾯的code中的tokens去除freq为0的 
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
        # vocab和其对应的tokens 
        vocab_tokenization[''.join(word_tokens)] = word_tokens
    return tokens_frequencies, vocab_tokenization

In [14]:
def measure_token_length(token): 
    # 如果token最后四个元素是 < / w > 
    if token[-4:] == '</w>':
        # 那就返回除了最后四个之外的⻓度再加上1(结尾) 
        return len(token[:-4]) + 1
    else:
        # 如果这个token⾥⾯没有结尾就直接返回当前⻓度
        return len(token)

In [12]:
# 对输入字符串进行分词
def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    
    # base case，没词进来了，那拆的结果就是空的
    if string == '':
        return []
    
    # 已有的sorted tokens没有了，那就真的没这个词了
    if sorted_tokens == []:
        return [unknown_token] * len(string)
    
    # 记录拆分结果
    string_tokens = []
    
    # iterate over all tokens to find match
    for i in range(len(sorted_tokens)):
        token = sorted_tokens[i]
        
        # ⾃定义⼀个正则，然后要把token⾥⾯包含句号的变成[.] 
        token_reg = re.escape(token.replace('.', '[.]'))
        
        # 在当前string⾥⾯遍历，找到每⼀个match token的开始和结束位置，⽐如string=good，然后token是o，输出[(2,2),(3,3)]
        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        # if no match found in the string, go to next token
        if len(matched_positions) == 0:
            continue
        # 因为要拆分这个词，匹配上的token把这个word拆开了，那就要拿到除了match部分之外的substring，所以这⾥要拿match的start 
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]
        substring_start_position = 0
        
        
        # 如果有匹配成功的话，就会进⼊这个循环
        for substring_end_position in substring_end_positions:
            # slice for sub-word
            substring = string[substring_start_position:substring_end_position]

            # tokenize this sub-word with tokens remaining 接着⽤substring匹配剩余的sorted token，因为刚就匹配了⼀个
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            
            # 先把sorted token⾥⾯匹配上的记下来
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)
        # tokenize the remaining string 去除前头的substring，去除已经匹配上的，后⾯还剩下substring_start_pos到结束的⼀段substring没看
 
        remaining_substring = string[substring_start_position:]
        # 接着匹配
 
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        break
    else: #在for循环正常结束时会执行，即都被continue了
        # return list of unknown token if no match is found for the string
        string_tokens = [unknown_token] * len(string)
        
    return string_tokens

In [15]:
"""
该函数⽣成⼀个所有token的列表，按其⻓度（第⼀键）和频率（第⼆键）排序。
EXAMPLE:
    token frequency dictionary before sorting: {'natural': 3, 'language':2, 'processing': 4, 'lecture': 4}
    sorted tokens: ['processing', 'language', 'lecture', 'natural']
 INPUT:
    token_frequencies: Dict[str, int] # Counter for token frequency
 OUTPUT:
    sorted_token: List[str] # Tokens sorted by length and frequency
 """
def sort_tokens(tokens_frequencies):
    # 对token_frequencies⾥⾯的东西，先进⾏⻓度排序，再进⾏频次，sorted是从低到⾼所以要reverse 
    sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item:(measure_token_length(item[0]),item[1]), reverse=True)

    # 然后只要tokens不要频次
    sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]
    return sorted_tokens

In [17]:
#display the vocab
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
#sort tokens by length and frequency
sorted_tokens = sort_tokens(tokens_frequencies)
print("Tokens =", sorted_tokens, "\n")
#print("vocab tokenization: ", vocab_tokenization)
sentence_1 = 'I like natural language processing!'
sentence_2 = 'I like natural languaaage processing!'
sentence_list = [sentence_1, sentence_2]
for sentence in sentence_list:
    print('==========')
    print("Sentence =", sentence)
    for word in sentence.split():
        word = word + "</w>"
        print('Tokenizing word: {}...'.format(word))
        if word in vocab_tokenization:
            print(vocab_tokenization[word])
        else:
            print(tokenize_word(string=word, sorted_tokens=sorted_tokens, unknown_token='</u>'))

Tokens = ['understanding</w>', 'algorithms</w>', 'language</w>', 'students</w>', 'subject</w>', 'develop</w>', 'ication', 'lation', 'ing,</w>', 'used</w>', 'inclu', 'ing.</w>', 'aims</w>', 'this</w>', 'main</w>', 'ding</w>', 'ation', 'for</w>', 'and</w>', 'The</w>', 'the</w>', 'use</w>', 'assi', 'gram', 'ing</w>', 'natu', 'nts</w>', 'in</w>', 'is</w>', 'to</w>', 'of</w>', 'pro', 'ver', 'ans', 'par', 'an</w>', 'ang', 'ion', 'ed</w>', 'for', 'e</w>', ',</w>', 's</w>', 'in', 'al', 'st', 'op', 'ic', 'de', 'on', 'or', 'me', 'ss', 't</w>', 'cl', 'ma', 'of', 'ec', 'ag', 'nt', 'ar', 'th', 'at', '.</w>', '</w>', 'e', 's', 't', 'r', 'c', 'p', 'l', 'h', 'm', 'a', 'o', '-', 'n', 'd', 'i', 'w', 'g', 'y', 'x', 'f', 'q', 'u', 'T', 'b', 'P', 'k'] 

Sentence = I like natural language processing!
Tokenizing word: I</w>...
['</u>', '</w>']
Tokenizing word: like</w>...
['l', 'i', 'k', 'e</w>']
Tokenizing word: natural</w>...
['natu', 'r', 'al', '</w>']
Tokenizing word: language</w>...
['language</w>']
Tok